In [3]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root= 'data/Planetoid', name='Cora', transform = NormalizeFeatures())

In [4]:
# Get some basic info about the dataset
print(f'Number of graphs {len(dataset)}')
print(f'Number of features {dataset.num_features}')
print(f'Number of classes {dataset.num_classes}')

Number of graphs 1
Number of features 1433
Number of classes 7


In [5]:
# There is only one graph in the dataset, use it as new data object
data = dataset[0]

In [6]:
# Gather some statistics about the graph.
print(data)
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Is Undirected: {data.is_undirected()}')

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Number of training nodes: 140
Is Undirected: True


In [7]:
data.x[0][:50]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [8]:
data.y

tensor([3, 4, 4,  ..., 3, 3, 3])

In [9]:
data.edge_index.t()

tensor([[ 633,    0],
        [1862,    0],
        [2582,    0],
        ...,
        [ 598, 2707],
        [1473, 2707],
        [2706, 2707]])

In [10]:
data.train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [11]:
data.test_mask

tensor([False, False, False,  ...,  True,  True,  True])

Training a Multi-layer Perception Network (MLP)

In [12]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
    def __init__(self,hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)
        
    def forward(self, x):
        x = self.lin1(x)
        x = F.relu(x)
        x = F.dropout(x,p=0.3, training = self.training)
        x= self.lin2(x)
        return x
    
model = MLP(hidden_channels = 16)
print(model)
        

MLP(
  (lin1): Linear(in_features=1433, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=7, bias=True)
)


In [13]:
#training
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01 ,weight_decay= 5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])        # Compute the loss solely based on the training nodes.
    loss.backward()
    optimizer.step()
    return loss

def test():
    model.eval()
    out = model(data.x)
    pred = out.argmax(dim=1)
    test_correct = pred[data.test_mask]==data.y[data.test_mask]
    test_acc = int(test_correct.sum())/int(data.test_mask.sum())
    return test_acc

for epoch in range(1,500):
    loss = train()
    print(f'Epoch: {epoch:03d}, loss: {loss:.4f}')

Epoch: 001, loss: 1.9617
Epoch: 002, loss: 1.9547
Epoch: 003, loss: 1.9483
Epoch: 004, loss: 1.9390
Epoch: 005, loss: 1.9300
Epoch: 006, loss: 1.9222
Epoch: 007, loss: 1.9078
Epoch: 008, loss: 1.9002
Epoch: 009, loss: 1.8887
Epoch: 010, loss: 1.8740
Epoch: 011, loss: 1.8656
Epoch: 012, loss: 1.8381
Epoch: 013, loss: 1.8267
Epoch: 014, loss: 1.8146
Epoch: 015, loss: 1.7998
Epoch: 016, loss: 1.7767
Epoch: 017, loss: 1.7554
Epoch: 018, loss: 1.7438
Epoch: 019, loss: 1.7151
Epoch: 020, loss: 1.6958
Epoch: 021, loss: 1.6806
Epoch: 022, loss: 1.6729
Epoch: 023, loss: 1.6272
Epoch: 024, loss: 1.5937
Epoch: 025, loss: 1.5751
Epoch: 026, loss: 1.5353
Epoch: 027, loss: 1.5123
Epoch: 028, loss: 1.4931
Epoch: 029, loss: 1.4727
Epoch: 030, loss: 1.4459
Epoch: 031, loss: 1.4165
Epoch: 032, loss: 1.3883
Epoch: 033, loss: 1.3637
Epoch: 034, loss: 1.3108
Epoch: 035, loss: 1.3153
Epoch: 036, loss: 1.2671
Epoch: 037, loss: 1.2581
Epoch: 038, loss: 1.2129
Epoch: 039, loss: 1.1697
Epoch: 040, loss: 1.1438


In [14]:
test_acc = test()
print(f'Test Accuracy : {test_acc:.4f}')

Test Accuracy : 0.6010


Training a Graph Neural Network (GNN)

In [15]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_channel):
        super(GCN,self).__init__()
        torch.manual_seed(42)
        
        self.conv1 = GCNConv(dataset.num_features, hidden_channel)
        self.conv2 = GCNConv(hidden_channel, hidden_channel)
        self.out = Linear(hidden_channel, dataset.num_classes)
        
    def forward(self, x, edge_index):
        x= self.conv1(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.3, training=self.training)
        
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.3, training=self.training)
        
        x = F.softmax(self.out(x), dim=1)
        
        return x
    
model = GCN(hidden_channel=16)
print(model)

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 16)
  (out): Linear(in_features=16, out_features=7, bias=True)
)


In [19]:
#Training
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x ,data.edge_index)
    loss = criterion(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    test_correct = pred[data.test_mask]==data.y[data.test_mask]
    test_acc = int(test_correct.sum())/int(data.test_mask.sum())
    return test_acc

losses = []
for epoch in range(1001):
    loss = train()
    losses.append(loss)
    if epoch%100 ==0:
        print(f'Epoch: {epoch: 03d},   loss: {loss: .4f}')
    

Epoch:  00,   loss:  1.2787
Epoch:  100,   loss:  1.2656
Epoch:  200,   loss:  1.2520
Epoch:  300,   loss:  1.2707
Epoch:  400,   loss:  1.2341
Epoch:  500,   loss:  1.2201
Epoch:  600,   loss:  1.2380
Epoch:  700,   loss:  1.2212
Epoch:  800,   loss:  1.2263
Epoch:  900,   loss:  1.2491
Epoch:  1000,   loss:  1.2566


In [20]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7640
